In [1]:
import numpy as np
import scipy.stats as stats
from scipy.integrate import quad
from scipy.special import beta as Beta
import pandas as pd
import itertools
import warnings
import scipy.optimize as optimize
from joblib import Parallel, delayed
from tqdm import trange
from collections import namedtuple
from scipy.signal import correlate, correlation_lags
from scipy.optimize import least_squares


Params = namedtuple('Parameters', ['mu', 'sigma', 'alpha', 'beta', 'p'])
str2float = lambda x: [float(k) for k in x[1:-1].split(',')]

# Parameter Estimation

 Parameter estimation for individuals within the (I)BN is based on the raw standing cycles (SC) and lying fractions (LF) distributions, while estimation of $u$ and $\delta$ for pairs of processes is based on the CCF shape, which is compared to a CCF predicted under the assumption of $u=1$ and $ \delta=0$.

## Density Functions in the IBN Model

Implementation of the SC density function in the IBN model. The density is given as
$$f_{SC}(c) = \frac{1-\pi}{1+\pi} f_C(c) + \frac{2\pi}{(1+\pi)B(\beta, \alpha)}\cdot\int_c^\infty  \frac{f_C(y)}{y^2} \left[ t \int_{\frac{c}{y}}^1  \xi(\alpha,\beta,x) dx + (y-t) \int_{1-\frac{c}{y}}^1  \xi(\alpha,\beta,x) dx \right] dy,$$
where $f_C$ is the PDF of $\mathcal{N}_+(\mu,\sigma)$, $B$ is the beta function and
$\xi(\alpha,\beta,x):= x^{\beta-3}(1-x)^{\alpha-1}$.

In [2]:
def f_SC(c: float, mu: float, sigma: float, alpha: float, beta: float, pi: float) -> float:
    """ Density function of standing cycles (SC) in the IBN model. """
    if c < 0:
        return 0
    
    f_C = lambda c: stats.truncnorm.pdf(c, -mu/sigma, np.inf, loc=mu, scale=sigma)

    # BN:
    if not pi:
        return f_C(c)
    
    # IBN:
    xi = lambda x: x**(beta - 3) * (1 - x)**(alpha - 1)

    summand1 = (1 - pi) / (1 + pi) * f_C(c)

    factor2 = 2 * pi / ((1 + pi) * Beta(beta, alpha))

    I = lambda v: quad(xi, v, 1)[0]
    summand2 = factor2 * quad(lambda y: f_C(y) / y**2 * (c* I(c/y) + (y - c) * I(1 - c/y)), c, np.inf)[0]  # Lemma 5.1

    return summand1 + summand2

Implementation of the LF density function in the IBN model. The density is given as
$$f_{LF}(c) = \frac{1-\pi}{1+\pi} f_F(c) + \frac{\pi}{1+\pi} + \frac{2\pi}{(1+\pi)c^2\cdot B(\beta, \alpha)} \int_{c^{-1}-1}^\infty x^{\beta -3} (1 + x)^{-\beta-\alpha} \cdot (x-c^{-1}+1)dx$$
for $c\in[0,1]$ and $f_{LF}(c)=0$ for $c\notin[0,1]$, where $f_F$ is the PDF of $Beta(\alpha,\beta)$ and $B$ is the beta function.

In [3]:
def f_LF(c: float, alpha: float, beta: float, pi: float) -> float:
    """ Density function of lying fractions (LF) in the IBN model. """
    if c < 0 or c > 1:
        return 0
    
    f_F = lambda x: stats.beta.pdf(x, alpha, beta)

    # BN:
    if not pi:
        return f_F(c)
    
    # IBN:
    z = 1/c - 1
    summand1 = (1 - pi) / (1 + pi) * f_F(c)

    summand2 = pi / (1 + pi)

    factor3 = 2 * pi / ((1 + pi) * c**2)

    I = lambda x: stats.betaprime.pdf(x, beta, alpha) * (1 - z/x) / x

    summand3 = factor3 * quad(I, z, np.inf)[0]

    return summand1 + summand2 + summand3

Additionally, start values for $\alpha$ and $\beta$ in the optimization algorithm inside the following parameter estimation functions are based on Method of Moments estimators for the Beta distribution, given by

$$
\hat{\alpha} = \bar{X} \left( \frac{\bar{X}(1 - \bar{X})}{s^2} - 1 \right)
$$

$$
\hat{\beta} = (1 - \bar{X}) \left( \frac{\bar{X}(1 - \bar{X})}{s^2} - 1 \right)
$$

where $\bar{X}$ is the sample mean and $s^2$ is the sample variance.

In [4]:
alpha_mom = lambda l: np.mean(l) * (np.mean(l) * (1 - np.mean(l)) / np.var(l) - 1)
beta_mom = lambda l: (1 - np.mean(l)) * (np.mean(l) * (1 - np.mean(l)) / np.var(l) - 1)

## Fitting function in the (I)BN

We fit the theoretical PDFs of SCs, $f_{SC}$ and LFs, $f_{LF}$ to their empirically observed histograms.

We use $K=20$ equally sized bins for the histogram of SC lengths. Additional simulation studies showed no notable improvement in model fit or parameter estimation with higher bin counts, indicating that 20 bins provide an effective balance between resolution and computational efficiency. These have a bin size of $\Delta^{(j)}_{SC} := {(\max_i c^{(j)}_i - \min_i c^{(j)})}/{K}$, where $c^{(j)}_1,\ldots,c^{(j)}_{n_j}$ denote the lengths of all fully observed SCs in animal $j$. For the histogram of LFs, we proceed analogously, i.e., $\Delta^{(j)}_{LF}:= {(\max_i f^{(j)}_i - \min_i f^{(j)})}/{K}$, where $f^{(j)}_1,\ldots,f^{(j)}_{n_j}$ denote the LFs of all fully observed SCs in animal $j$. This yields bin-midpoints $b^{SC,(j)}_{k}$ and $b^{LF}_{k}$, $k=1,\ldots,K,$ as
$$b^{SC,(j)}_k := \min_i c^{(j)}_i + (k - 0.5) \cdot \Delta^{(j)}_{SC} \quad \text{ and } \quad b^{LF,(j)}_k := \min_i f^{(j)}_i + (k - 0.5) \cdot \Delta^{(j)}_{LF}.$$
The corresponding bin counts, normed to a unit area of the histogram, are then
$d^{SC,(j)}_k := (|\{i: b^{SC,(j)}_k - \Delta^{(j)}_{SC} \leq c_i < b^{SC}_k\}| + \delta_K(k))/(n_j\Delta^{(j)}_{SC})$, where "$<$" is replaced by "$\leq$" for $k=K$, and analogously for $d^{LF,(j)}_{k}$.


Then $f_{SC}$ and $f_{LF}$ are fitted jointly using a non-linear-least squares approach, i.e. parameters are estimated by minimizing the term
$$T_j:=\sum_{k=1}^{K} \left(\Delta^{(j)}_{SC}\right)^{2} \cdot\left(f_{SC}(b^{SC}_k) - d^{SC}_k\right)^2 + \Delta^2_{LF} \cdot \left(f_{LF}(b^{LF}_k) - d^{LF}_k\right)^2.$$

$T_j$ is minimzed using non-linear-least-squares approach based on the Trust Region Reflective algorithm for minimization. An initial guess for the parameters is given by
$$\mu_0 = \bar c + 30, \sigma_0 = sd(f), \alpha_0=\hat\alpha_{MOM},\text{ and } \beta=\hat\beta_{MOM}.$$
Lower and upper bounds on parameters are given by $(0, 660)$ for $\mu, \sigma$ and $(0, 1000)$ for $\alpha, \beta$.

In [5]:
def _fit_BN(alpha_mom, beta_mom,
			x_scs, y_scs, factor_scs,
			x_lfs, y_lfs, factor_lfs):
	
	eps = np.finfo(np.float32).eps
	
	# Combine the x and y values for standing cycles (SCs) and lying fractions (LFs)
	x = np.array([b for b in x_scs] + [b for b in x_lfs])
	y = np.array([factor_scs*b for b in y_scs]  + [factor_lfs*b for b in y_lfs])

	def compute_SC(x, mu, sigma, a, b, factor_scs):
		# Compute the density function for standing cycles (SCs)
		return factor_scs * f_SC(x, mu, sigma, a, b, 0)

	def compute_LF(x, a, b, factor_lfs):
		# Compute the density function for lying fractions (LFs)
		return factor_lfs * f_LF(x, a, b, 0)

	def f(_x, mu, sigma, a, b):
		# Combine the density functions for SCs and LFs
		split_index = len(_x) // 2
		y1 = Parallel(n_jobs=-1)(delayed(compute_SC)(x, mu, sigma, a, b, factor_scs) for x in _x[:split_index])
		y2 = Parallel(n_jobs=-1)(delayed(compute_LF)(x, a, b, factor_lfs) for x in _x[split_index:])
		return np.concatenate((y1, y2))

	with warnings.catch_warnings(record=True) as w:
		warnings.simplefilter("always")
		scs = list(x_scs)
		# Fit the combined density function to the data using curve fitting
		result = optimize.curve_fit(f, x, y, p0=[np.mean(scs) + 30, np.std(scs), alpha_mom, beta_mom],
									bounds=(eps, (660, 660, 1_000, 1_000)), diff_step=0.01)[0]
		if any(issubclass(w.category, UserWarning) for w in w):
			warning_occured = True
		else:
			warning_occured = False
		optimal_value = np.sum((f(x, *result) - y)**2)

	return *result, optimal_value, float(warning_occured)  # Return the optimized parameters, optimal value, and warning flag


def fit_BN(scs, lfs, bins=40):
	# Compute Method of Moments estimators for alpha and beta
	alpha_moms = alpha_mom(lfs)
	beta_moms = beta_mom(lfs)

	# Compute histograms for SCs and LFs
	b_y, b_x = np.histogram(scs, bins=bins, density=True)
	dx = b_x[1] - b_x[0]
	factor_scs = dx
	x_scs = b_x[:-1] + dx/2
	y_scs = b_y.copy()


	b_y, b_x = np.histogram(lfs, bins=bins, density=True)
	dx = b_x[1] - b_x[0]
	factor_lfs = dx
	x_lfs = b_x[:-1] + dx/2
	y_lfs = b_y.copy()

	# Call the internal function to fit the BN model to the data
	return _fit_BN(alpha_moms, beta_moms, x_scs, y_scs, factor_scs, x_lfs, y_lfs, factor_lfs) 

The IBN case is analogous to the BN case, except for the additional parameter $\pi$. The minimization is performed twice, once with initial guess $\pi_0 = 0.25$ (and other parameters as described above) and once with $\pi_0 = 0.75$. Then, the result with the lowest achieved error is returned.

In [6]:
def fit_IBN(scs, lfs, bins=40):
	eps = np.finfo(np.float32).eps

	alpha_moms = alpha_mom(lfs)
	beta_moms = beta_mom(lfs)

	b_y, b_x = np.histogram(scs, bins=bins, density=True)
	dx = b_x[1] - b_x[0]
	factor_scs = dx
	x_scs = b_x[:-1] + dx/2
	y_scs = b_y.copy()

	b_y, b_x = np.histogram(lfs, bins=bins, density=True)
	dx = b_x[1] - b_x[0]
	factor_lfs = dx
	x_lfs = b_x[:-1] + dx/2
	y_lfs = b_y.copy()

	
	x = np.array([b for b in x_scs] + [b for b in x_lfs])
	y = np.array([factor_scs*b for b in y_scs]  + [factor_lfs*b for b in y_lfs])

	def compute_SC(x, mu, sigma, a, b, p, factor_scs):
		return factor_scs * f_SC(x, mu, sigma, a, b, p/100)

	def compute_LF(x, a, b, p, factor_lfs):
		return factor_lfs * f_LF(x, a, b, p/100)

	def f(_x, mu, sigma, a, b, p):
		split_index = len(_x) // 2
		y1 = Parallel(n_jobs=-1)(delayed(compute_SC)(x, mu, sigma, a, b, p, factor_scs) for x in _x[:split_index])
		y2 = Parallel(n_jobs=-1)(delayed(compute_LF)(x, a, b, p, factor_lfs) for x in _x[split_index:])
		return np.concatenate((y1, y2))

	results = {}
	for p0 in [0.25, 0.75]:
		with warnings.catch_warnings(record=True) as w:
			warnings.simplefilter("always")
			scs = list(x_scs)
			result = optimize.curve_fit(f, x, y, p0=[np.mean(scs) + 30, np.std(scs), alpha_moms, beta_moms, p0*100],
										bounds=(eps, (660, 660, 1_000, 1_000, 100)), diff_step=0.01)[0]
			if any(issubclass(w.category, UserWarning) for w in w):
				warning_occured = True
			else:
				warning_occured = False
			optimal_value = np.sum((f(x, *result) - y)**2)
			results[p0] = (*result[:4], result[4]/100, optimal_value, float(warning_occured))

	optimal_values = {key: results[key][-2] for key in results}
	best_fit = min(optimal_values, key=optimal_values.get)
	return *results[best_fit], best_fit  # 8 values

## Fitting function in the S-IBN

An analogous fitting approach is applied for the S-IBN. We first fit the theoretical PDFs of SCs and LFs to their respective histograms as described above. Second, in order to estimate the parameters $\delta$ and $u$, we predict the CCF $\rho^0_Z(h)$ with parameters $\hat\mu, \hat\sigma, \hat\alpha_1, \hat \alpha_2, \hat \beta_1, \hat \beta_2, \hat \pi_1,$ and $\hat \pi_2$ for an S-IBN with $u=1, \delta=0$ and then fit the function $u \cdot \rho_Z(h+\delta)$ to the observed CCF using nonlinear least squares.

In [7]:
def fit_SIBN(scs1, lfs1, scs2, lfs2, bins=20):
	""" Fit all parameters except for delta and u of the SIBN model to the data (SCs and LFs of both animals). """
	eps = np.finfo(np.float32).eps

	alpha_moms1 = alpha_mom(lfs1)
	beta_moms1 = beta_mom(lfs1)
	alpha_moms2 = alpha_mom(lfs2)
	beta_moms2 = beta_mom(lfs2)

	b_y, b_x = np.histogram(scs1, bins=bins, density=True)
	dx = b_x[1] - b_x[0]
	factor_scs1 = dx
	x_scs1 = b_x[:-1] + dx/2
	y_scs1 = b_y.copy()

	b_y, b_x = np.histogram(lfs1, bins=bins, density=True)
	dx = b_x[1] - b_x[0]
	factor_lfs1 = dx
	x_lfs1 = b_x[:-1] + dx/2
	y_lfs1 = b_y.copy()

	b_y, b_x = np.histogram(scs2, bins=bins, density=True)
	dx = b_x[1] - b_x[0]
	factor_scs2 = dx
	x_scs2 = b_x[:-1] + dx/2
	y_scs2 = b_y.copy()

	b_y, b_x = np.histogram(lfs2, bins=bins, density=True)
	dx = b_x[1] - b_x[0]
	factor_lfs2 = dx
	x_lfs2 = b_x[:-1] + dx/2
	y_lfs2 = b_y.copy()
	
	x = np.array([b for b in x_scs1] + [b for b in x_scs2] + [b for b in x_lfs1] + [b for b in x_lfs2])
	y = np.array([factor_scs1*b for b in y_scs1] + [factor_scs2*b for b in y_scs2] + [factor_lfs1*b for b in y_lfs1] + [factor_lfs2*b for b in y_lfs2])

	def compute_SC(x, mu, sigma, a, b, p, factor_scs):
		return factor_scs * f_SC(x, mu, sigma, a, b, p/100)

	def compute_LF(x, a, b, p, factor_lfs):
		return factor_lfs * f_LF(x, a, b, p/100)

	def f(_x, mu, sigma, a1, b1, p1, a2, b2, p2):
		split_index = len(_x) // 4
		y1 = Parallel(n_jobs=-1)(delayed(compute_SC)(x, mu, sigma, a1, b1, p1, factor_scs1) for x in _x[:split_index])
		y2 = Parallel(n_jobs=-1)(delayed(compute_SC)(x, mu, sigma, a2, b2, p2, factor_scs2) for x in _x[split_index:2*split_index])
		y3 = Parallel(n_jobs=-1)(delayed(compute_LF)(x, a1, b1, p1, factor_lfs1) for x in _x[2*split_index:3*split_index])
		y4 = Parallel(n_jobs=-1)(delayed(compute_LF)(x, a2, b2, p2, factor_lfs2) for x in _x[3*split_index:])
		return np.concatenate((y1, y2, y3, y4))

	results = {}
	for p0_0, p1_0 in itertools.product([0.25, 0.75], repeat=2):
		with warnings.catch_warnings(record=True) as w:
			warnings.simplefilter("always")
			scs = list(x_scs1) + list(x_scs2)
			result = optimize.curve_fit(f, x, y, p0=[np.mean(scs) + 30, np.std(scs), alpha_moms1, beta_moms1, p0_0*100, alpha_moms2, beta_moms2, p1_0*100],
										bounds=(eps, (660, 660, 1_000, 1_000, 100, 1_000, 1_000, 100)), diff_step=0.01, xtol=1e-1, ftol=1e-1)[0]
			if any(issubclass(w.category, UserWarning) for w in w):
				warning_occured = True
			else:
				warning_occured = False
			optimal_value = np.sum((f(x, *result) - y)**2)
			results[(p0_0, p1_0)] = (*result[:4], result[4]/100, *result[5:7], result[7]/100, optimal_value, float(warning_occured))

	optimal_values = {key: results[key][-2] for key in results}
	best_fit = min(optimal_values, key=optimal_values.get)
	return *results[best_fit], *best_fit  # 11 values

To fit $\delta$ and $u$ we need some more utility functions to compute both the empirical and the theoretical CCFs, which are then fitted using least-squares. 

Funnctions to imitate pre-processing:

In [8]:
def remove_small_cycles(binary_list, length=300):
	if not isinstance(binary_list, np.ndarray):
		binary_list = np.array(binary_list)
	nr_removed_phases = 0
	while True:
		indices = np.concatenate(([0], np.where(np.diff(binary_list) != 0)[0] + 1))
		lengths = np.diff(np.concatenate(([0], indices, [len(binary_list)])))[1:]
		if lengths.min() >= length:
			break
		index = np.where(lengths < length)[0][0]
		binary_list[indices[index]:indices[index]+lengths[index]+1] = 1 - binary_list[indices[index]]
		nr_removed_phases += 1
	return binary_list, nr_removed_phases

Function to simulate shared nights:

In [9]:
def generate_shared_night(p1, p2, k=100, max_length=11*60*60, remove_length=300, delta=0) -> tuple[list[int]]:
	assert p1.mu == p2.mu and p1.sigma == p2.sigma
	night1, night2 = [], []
	while len(night1) <= (max_length + 10*p1.mu)  or len(night2) <= (max_length + 10*p1.mu):
		C = stats.truncnorm.rvs(-p1.mu/p1.sigma, np.inf, loc=p1.mu, scale=p1.sigma, size=k)
		F1 = iter(stats.beta.rvs(p1.alpha, p1.beta, size=k))
		F2 = iter(stats.beta.rvs(p2.alpha, p2.beta, size=k))
		U1 = stats.uniform.rvs(size=(k, 2))
		U_mins1 = iter(np.min(U1, axis=1))
		U_maxs1 = iter(np.max(U1, axis=1))
		U2 = stats.uniform.rvs(size=(k, 2))
		U_mins2 = iter(np.min(U2, axis=1))
		U_maxs2 = iter(np.max(U2, axis=1))
		P1 = stats.bernoulli.rvs(p1.p, size=k)
		P2 = stats.bernoulli.rvs(p2.p, size=k)

		t0 = round(stats.uniform.rvs(0, C[0], size=1)[0])
		C = iter(C)


		for interruption1, interruption2 in zip(P1, P2):
			c, f1, f2 = round(next(C)), next(F1), next(F2)
			s1, s2 = round(c * (1 - f1)), round(c * (1 - f2))
			l1, l2 = c - s1, c - s2

			if not interruption1:
				night1 += [1] * s1 + [0] * l1
			else:
				u_min, u_max = next(U_mins1), next(U_maxs1)
				s11 = round(s1 * u_min)
				l11 = round(s1 * (u_max - u_min))
				s12 = s1 - s11 - l11
				assert s12 >= 0

				night1 += [1] * s11 + [0] * l11 + [1] * s12 + [0] * l1
			if not interruption2:
				night2 += [1] * s2 + [0] * l2
			else:
				u_min, u_max = next(U_mins2), next(U_maxs2)
				s21 = round(s2 * u_min)
				l21 = round(s2 * (u_max - u_min))
				s22 = s2 - s21 - l21
				assert s22 >= 0

				night2 += [1] * s21 + [0] * l21 + [1] * s22 + [0] * l2
	night1 = night1[t0:]
	night2 = night2[t0:]
	if delta > 0:
		night1 = night1[delta:]
	elif delta < 0:
		night2 = night2[delta:]
	night1 = night1[:max_length]
	night2 = night2[:max_length]

	night1_trimmed, num_removed_phases1 = remove_small_cycles(night1, remove_length)
	night2_trimmed, num_removed_phases2 = remove_small_cycles(night2, remove_length)
	return night1, night1_trimmed, night2, night2_trimmed, num_removed_phases1, num_removed_phases2

In [10]:
def compute_E_at_lag(n1, n2, lag):
    if lag > 0:
        E = np.mean(n1[:-lag] * n2[lag:])
    elif lag < 0:
        E = np.mean(n1[-lag:] * n2[:lag])
    else:
        E = np.mean(n1 * n2)
    return E

def compute_E(p1: Params, p2: Params, n=100, max_lag=100):
    n1, _, n2, _, _, _ = generate_shared_night(p1, p2, max_length=n, k=round(np.log(n)))
    print(len(n1))
    lags = np.arange(-max_lag, max_lag+1)
    E = np.array([compute_E_at_lag(np.array(n1), np.array(n2), lag) for lag in lags])
    return E

def compute_CCF(p1: Params, p2: Params, n=100, max_lag=100):
    E = compute_E(p1, p2, n=n, max_lag=max_lag)

    mu_X1 = p1.beta / (p1.alpha + p1.beta)
    mu_X2 = p2.beta / (p2.alpha + p2.beta)

    mu_1 = (1 - p1.p/3) * mu_X1
    mu_2 = (1 - p2.p/3) * mu_X2

    var_1 = mu_X1 * (1 - p1.p/3) * (1 - mu_X1 * (1 - p1.p/3))
    var_2 = mu_X2 * (1 - p2.p/3) * (1 - mu_X2 * (1 - p2.p/3))

    CCF = (E - mu_1 * mu_2) / np.sqrt(var_1 * var_2)
    return CCF

Function to compute the empirical CCF of two nights:

In [11]:
def compute_ccf_night(x, y, mode='full', max_lag=None):
	# Calculate raw cross-correlation
	ccf = correlate(x - np.mean(x), y - np.mean(y), mode, method='fft')
	
	# Normalize the raw cross-correlation
	ccf /= np.sqrt(np.sum((x - np.mean(x))**2) * np.sum((y - np.mean(y))**2))
	if max_lag is not None:
		ccf = ccf[len(ccf)//2-max_lag:len(ccf)//2+max_lag+1]
	return ccf

Function to compute the theoretical CCF of two animals:

In [12]:
def ccf_pair_theo(p1, p2, max_lag=6*60, n=100):
	""" Compute the theoretical cross-correlation function (CCF) for a pair of parameters. """
	n1, _, n2, _, _, _ = generate_shared_night(p1, p2, n)
	ccf = compute_ccf_night(n1, n2, max_lag=max_lag)
	return ccf


def tf(u, delta, ccf):
	""" Shift the cross-correlation function (CCF) by delta and scale it by u. """
	k = len(ccf) // 2
	lags = np.arange(-k, k+1)
	shifted_ccf = np.nan_to_num(np.interp(lags + delta, lags, ccf))
	return u * shifted_ccf


def fit_delta_u(emp_ccf, theo_ccf):
	""" Fit the shift delta and scale u of the theoretical CCF to the empirical CCF. """
	k = len(emp_ccf) // 2
	lags = np.arange(-k, k+1)
	assert len(emp_ccf) == len(theo_ccf) == len(lags), f'{len(emp_ccf)} != {len(theo_ccf)} != {len(lags)}'

	def f(u, delta):
		shifted_ccf_theo = np.nan_to_num(np.interp(lags + delta, lags, theo_ccf))
		y = np.sum((u * shifted_ccf_theo - emp_ccf)[k-4*60**2:k + 4 * 60**2]**2)
		return y

	cost = np.inf
	for delta in range(-1200, 1200, 6):
		res = least_squares(f, [.5], args=(delta,), bounds=([0], [1]))
		if res.cost < cost:
			cost = res.cost
			best_res = res
			best_delta = delta
	print(best_delta, best_res.x)
	return best_delta, best_res.x[0]

# Simulation of Parameter Estimation

## Simulation of Parameter Estimation in the IBN

Funnctions to simulate nights:

In [13]:
def generate_night(p1, k=100, max_length=11*60*60, remove_length=300) -> tuple[list[int]]:
	night = []
	while len(night) <= (max_length + 10*p1.mu):
		C = stats.truncnorm.rvs(-p1.mu/p1.sigma, np.inf, loc=p1.mu, scale=p1.sigma, size=k)
		F1 = iter(stats.beta.rvs(p1.alpha, p1.beta, size=k))
		U1 = stats.uniform.rvs(size=(k, 2))
		U_mins1 = iter(np.min(U1, axis=1))
		U_maxs1 = iter(np.max(U1, axis=1))
		P1 = stats.bernoulli.rvs(p1.p, size=k)

		t0 = round(stats.uniform.rvs(0, C[0], size=1)[0])
		C = iter(C)

		for interruption1 in P1:
			c, f1 = round(next(C)), next(F1)
			s1 = round(c * (1 - f1))
			l1 = c - s1

			if not interruption1:
				night += [1] * s1 + [0] * l1
			else:
				u_min, u_max = next(U_mins1), next(U_maxs1)
				s11 = round(s1 * u_min)
				l11 = round(s1 * (u_max - u_min))
				s12 = s1 - s11 - l11
				assert s12 >= 0

				night += [1] * s11 + [0] * l11 + [1] * s12 + [0] * l1
			
	night = night[t0:]
	night = night[:max_length]

	night_trimmed, num_removed_phases = remove_small_cycles(night, remove_length)
	return night, night_trimmed, num_removed_phases


def trim_partial_cycles(n):
	if n[0]:
		n = 1 - np.trim_zeros(1-n, 'f')
	n = np.trim_zeros(n, 'fb')
	n = 1 - np.trim_zeros(1-n, 'b')
	return n


def get_scs_lfs(n):
	n = trim_partial_cycles(n)
	if n.size == 0:
		return [], []
	one_seq = np.split(n, np.where(np.diff(n) != 0)[0] + 1)
	if len(one_seq) == 0:
		return [], []
	one_seq = np.array([len(seq) for seq in one_seq if seq[0] == 1])
	zero_seq = np.split(n, np.where(np.diff(n) != 0)[0] + 1)
	if len(zero_seq) == 0:
		return [], []
	zero_seq = np.array([len(seq) for seq in zero_seq if seq[0] == 0])

	scs = one_seq + zero_seq
	lfs = zero_seq / scs

	return scs, lfs

Function to run Simulations:

In [14]:
def run_simulation_IBN(BINS, RUNS, NIGHTS, P):	
	p = Params(144*60, 40*60, 5.25, 2.52, P)

	nights = np.zeros((RUNS, NIGHTS, 11*60*60), dtype=np.uint8)
	nights_trimmed = np.zeros((RUNS, NIGHTS, 11*60*60), dtype=np.uint8)
	num_removed_phases = np.zeros((RUNS, NIGHTS), dtype=np.uint16)

	for i in trange(RUNS):
		for j in range(NIGHTS):
			nights[i][j], nights_trimmed[i][j], num_removed_phases[i][j] = generate_night(p)

	scs, lfs = [], []
	scs_trimmed, lfs_trimmed = [], []
	for i in trange(RUNS):
		scs_, lfs_, = [], []
		scs_trimmed_, lfs_trimmed_ = [], []

		for j in range(NIGHTS):
			scs__, lfs__ = get_scs_lfs(nights[i][j])
			scs_.extend([s/60 for s in scs__])
			lfs_.extend(lfs__)
		scs.append(scs_)
		lfs.append(lfs_)

		for j in range(NIGHTS):
			scs__, lfs__ = get_scs_lfs(nights_trimmed[i][j])
			scs_trimmed_.extend([s/60 for s in scs__])
			lfs_trimmed_.extend(lfs__)
		scs_trimmed.append(scs_trimmed_)
		lfs_trimmed.append(lfs_trimmed_)
	

	for i in trange(RUNS, desc='Fitting'):
		results = fit_IBN(scs[i], lfs[i], BINS)
		with open(f'IBN_fitting_simulation_results_{RUNS}runs_{BINS}bins_{NIGHTS}nights.csv', 'a') as f:
			f.write(f'{i},{RUNS},{BINS},{NIGHTS},{",".join(map(str, p))},{",".join(map(str, results))},0,False,{len(scs[i])}\n')
		results = fit_IBN(scs_trimmed[i], lfs_trimmed[i], BINS)
		with open(f'IBN_fitting_simulation_results_{RUNS}runs_{BINS}bins_{NIGHTS}nights.csv', 'a') as f:
			f.write(f'{i},{RUNS},{BINS},{NIGHTS},{",".join(map(str, p))},{",".join(map(str, results))},{str(np.sum(num_removed_phases[i]))},True,{len(scs_trimmed[i])}\n')


Run simulations:

In [ ]:
bins = [10, 20, 30, 40, 50, 60]
nights = [10, 20, 30, 40, 50, 80, 90, 100]
pis = [0, 0.3, 0.5, 0.7, 0.9]

for b, n, p in itertools.product(bins, nights, pis):
    run_simulation_IBN(b, 100, n, p)

## Simulation of paramter estimation in the S-IBN

Function to run simulation:

In [16]:
def compute_ccf(x, y, maxlags=None):
	"""Compute the cross-correlation function."""
	if maxlags is None:
		maxlags = len(x) - 1
	correlation = correlate(x - np.mean(x), y - np.mean(y), mode='full', method='fft')
	lags = correlation_lags(len(x), len(y), mode='full')
	correlation = correlation / (len(x) * np.std(x) * np.std(y))
	midpoint = len(correlation) // 2
	return correlation[midpoint-maxlags:midpoint+maxlags+1], lags[midpoint-maxlags:midpoint+maxlags+1]

def run_simulation_SIBN(BINS, RUNS, NIGHTS, P1, P2, DELTA, U, REMOVE, MAXLAGS=18000):
	p1 = Params(144*60, 40*60, 5.25, 2.52, P1)
	p2 = Params(144*60, 40*60, 5.25, 2.52, P2)

	nights1, nights2 = np.zeros((RUNS, NIGHTS, 11*60*60), dtype=np.uint8), np.zeros((RUNS, NIGHTS, 11*60*60), dtype=np.uint8)
	nights1_trimmed, nights2_trimmed = np.zeros((RUNS, NIGHTS, 11*60*60), dtype=np.uint8), np.zeros((RUNS, NIGHTS, 11*60*60), dtype=np.uint8)
	num_removed_phases1, num_removed_phases2 = np.zeros((RUNS, NIGHTS), dtype=np.uint16), np.zeros((RUNS, NIGHTS), dtype=np.uint16)

	for i in trange(RUNS):
		for j in range(int(U * NIGHTS)):
			nights1[i][j], nights1_trimmed[i][j], nights2[i][j], nights2_trimmed[i][j], num_removed_phases1[i][j], num_removed_phases2[i][j] = generate_shared_night(p1, p2, delta=DELTA, remove_length=REMOVE)
		for j in range(int(U * NIGHTS), NIGHTS):
			nights1[i][j], nights1_trimmed[i][j], num_removed_phases1[i][j] = generate_night(p1, remove_length=REMOVE)
			nights2[i][j], nights2_trimmed[i][j], num_removed_phases2[i][j] = generate_night(p2, remove_length=REMOVE)
	scs1, lfs1, scs2, lfs2 = [], [], [], []
	scs1_trimmed, lfs1_trimmed, scs2_trimmed, lfs2_trimmed = [], [], [], []

	np.save(f'CCF_SIBN_trimmed_untrimmed_fit_simulation_results_nights1_{RUNS}runs_{NIGHTS}nights_{BINS}bins_{U}u_{DELTA}delta.npy', nights1)
	np.save(f'CCF_SIBN_trimmed_untrimmed_fit_simulation_results_nights2_{RUNS}runs_{NIGHTS}nights_{BINS}bins_{U}u_{DELTA}delta.npy', nights2)
	np.save(f'CCF_SIBN_trimmed_untrimmed_fit_simulation_results_nights1_trimmed_{RUNS}runs_{NIGHTS}nights_{BINS}bins_{U}u_{DELTA}delta.npy', nights1_trimmed)
	np.save(f'CCF_SIBN_trimmed_untrimmed_fit_simulation_results_nights2_trimmed_{RUNS}runs_{NIGHTS}nights_{BINS}bins_{U}u_{DELTA}delta.npy', nights2_trimmed)

	for i in trange(RUNS):
		scs1_, lfs1_, scs2_, lfs2_ = [], [], [], []
		for j in range(NIGHTS):
			scs, lfs = get_scs_lfs(nights1[i][j])
			scs1_.extend([s/60 for s in scs])
			lfs1_.extend(lfs)
			scs, lfs = get_scs_lfs(nights2[i][j])
			scs2_.extend([s/60 for s in scs])
			lfs2_.extend(lfs)
		scs1.append(scs1_)
		lfs1.append(lfs1_)
		scs2.append(scs2_)
		lfs2.append(lfs2_)

		scs1_, lfs1_, scs2_, lfs2_ = [], [], [], []
		for j in range(NIGHTS):
			scs, lfs = get_scs_lfs(nights1_trimmed[i][j])
			scs1_.extend([s/60 for s in scs])
			lfs1_.extend(lfs)
			scs, lfs = get_scs_lfs(nights2_trimmed[i][j])
			scs2_.extend([s/60 for s in scs])
			lfs2_.extend(lfs)
		scs1_trimmed.append(scs1_)
		lfs1_trimmed.append(lfs1_)
		scs2_trimmed.append(scs2_)
		lfs2_trimmed.append(lfs2_)

	theoretical_ccfs_untrimmed = np.zeros(RUNS, 2 * MAXLAGS + 1)
	theoretical_ccfs_trimmed = np.zeros(RUNS, 2 * MAXLAGS + 1)
	avg_ccfs_untrimmed = np.zeros((RUNS, 2*MAXLAGS+1))
	avg_ccfs_trimmed = np.zeros((RUNS, 2*MAXLAGS+1))
	lags = np.zeros(2*MAXLAGS+1)



	for i in trange(RUNS, desc='Fitting'):
		results = fit_SIBN(scs1[i], lfs1[i], scs2[i], lfs2[i], BINS)
		results_trimmed = fit_SIBN(scs1_trimmed[i], lfs1_trimmed[i], scs2_trimmed[i], lfs2_trimmed[i], BINS)

		# Compute theoretical CCF:
		p1 = Params(*results[:5])
		p2 = Params(*results[:2], *results[5:8])
		
		ccf = compute_CCF(p1, p2, max_lag=MAXLAGS)
		theoretical_ccfs_untrimmed[i] = ccf

		p1 = Params(*results_trimmed[:5])
		p2 = Params(*results_trimmed[:2], *results_trimmed[5:8])
		
		ccf = compute_CCF(p1, p2, max_lag=MAXLAGS)
		theoretical_ccfs_trimmed[i] = ccf

		# Compute empirical CCF:
		ccfs_untrimmed_run = np.zeros((NIGHTS, 2*MAXLAGS+1))
		ccfs_trimmed_run = np.zeros((NIGHTS, 2*MAXLAGS+1))

		for night in range(NIGHTS):
			# Untrimmed nights
			ccf, _ = compute_ccf(nights1[i, night], nights2[i, night], maxlags=MAXLAGS)
			ccfs_untrimmed_run[night] = ccf

			# Trimmed nights
			ccf, _ = compute_ccf(nights1_trimmed[i, night], nights2_trimmed[i, night], maxlags=MAXLAGS)
			ccfs_trimmed_run[night] = ccf

		# Average CCFs over nights for this run
		avg_ccfs_untrimmed[i] = np.mean(ccfs_untrimmed_run, axis=0)
		avg_ccfs_trimmed[i] = np.mean(ccfs_trimmed_run, axis=0)

		# Compute delta and u
		delta_untrimmed, u_untrimmed = fit_delta_u(avg_ccfs_untrimmed[i], theoretical_ccfs_untrimmed[i])
		delta_trimmed, u_trimmed = fit_delta_u(avg_ccfs_trimmed[i], theoretical_ccfs_trimmed[i])


		with open(f'CCF_SIBN_trimmed_untrimmed_fit_simulation_results_{RUNS}runs_{BINS}bins_{NIGHTS}nights_{DELTA}delta_{U}u.csv', 'a') as f:
			f.write(f'{i},{RUNS},{BINS},{NIGHTS},{",".join(map(str, P1))},{",".join(map(str, P2))},{DELTA},{U},{",".join(map(str, results))},0,False,{len(scs1[i])},{len(scs2[i])},{delta_untrimmed},{u_untrimmed}\n')
		with open(f'CCF_SIBN_trimmed_untrimmed_fit_simulation_results_{RUNS}runs_{BINS}bins_{NIGHTS}nights_{DELTA}delta_{U}u.csv', 'a') as f:
			f.write(f'{i},{RUNS},{BINS},{NIGHTS},{",".join(map(str, P1))},{",".join(map(str, P2))},{DELTA},{U},{",".join(map(str, results))},{str(np.sum(num_removed_phases1[i]))},{str(np.sum(num_removed_phases1[i]))},True,{len(scs1_trimmed[i])},{len(scs2_trimmed[i])},{delta_trimmed},{u_trimmed}\n')


Run simulations

In [ ]:
nights = [10, 30, 50, 70, 90]
deltas = [0, 60, 300]
us = [0, 0.5, 0.75, 1]

for n, d, u in itertools.product(nights, deltas, us):
    run_simulation_SIBN(40, 100, n, 0, 0, d, u, 300)
    run_simulation_SIBN(40, 100, n, 0, 0.5, d, u, 300)
    run_simulation_SIBN(40, 100, n, 0, 0.9, d, u, 300)
    run_simulation_SIBN(40, 100, n, 0.3, 0.9, d, u, 300)

# Data analysis

Load data 

In [ ]:
data = pd.read_csv('data.csv')
nights = pd.read_csv('nights.csv')

## Data description

Filter animals that have less than 30 fully observed SCs:

In [18]:
data.query('nr_scs < 30')

,sex,species,id,scs,lfs,starts,ends,ly_starts,scs_mean,scs_std,...,mu,sigma,alpha,beta,pi,nr_scs,nr_nights,partner,nr_scs_per_night,age
95,male,Oryx dammah,96,"[147.58333333333334, 144.78333333333333, 183.4...","[0.6948616600790514, 0.8162771958098308, 0.951...","[125.41666666666667, 376.25, 109.0833333333333...","[273.0, 521.0333333333333, 292.48333333333335,...","[170.45, 402.85, 117.95, 330.6333333333333, 16...",136.277778,56.345254,...,126.595866,51.895132,15.295126,1.675219,1.050116e-01,21,23,-1,"[1, 1, 2, 0, 2, 1, 0, 0, 1, 0, 0, 2, 0, 0, 0, ...",adult
170,female,Equus quagga,171,"[131.48333333333332, 191.21666666666667, 175.1...","[0.5865128660159716, 0.7132397803538744, 0.732...","[150.15, 281.6333333333333, 472.85, 144.433333...","[281.6333333333333, 472.85, 647.9666666666667,...","[204.51666666666668, 336.46666666666664, 519.7...",176.489744,45.354351,...,191.428873,48.581164,7.780506,3.964330,1.202961e-09,26,9,-1,"[3, 3, 3, 2, 4, 3, 2, 2, 4]",adult
174,female,Equus quagga,175,"[335.06666666666666, 97.88333333333334, 419.76...","[0.018802228412256268, 0.11323003575685339, 0....","[127.75, 131.01666666666668, 228.9, 120.75, 12...","[462.81666666666666, 228.9, 648.6666666666666,...","[456.51666666666665, 217.81666666666666, 643.4...",202.154167,123.103883,...,174.920073,3.565168,13.979368,833.182869,5.435323e-01,8,27,-1,"[0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 1, 0, 1, ...",adult


In [19]:
data = data.query('nr_scs >= 30')
nights = nights[nights['id'].isin(data['id'])]

In [20]:
data.sex.value_counts()

female    129
male       60
Name: sex, dtype: int64

In [13]:
data.species.value_counts()


Equus quagga                32
Tragelaphus oryx            25
Kobus ellipsiprymnus        19
Connochaetes taurinus       17
Damaliscus pygargus         14
Okapia johnstoni            11
Tragelaphus eurycerus       10
Equus grevyi                10
Addax nasomaculatus          8
Syncerus c. nanus            7
Equus zebra                  7
Tragelaphus strepsiceros     6
Hippotragus equinus          6
Oryx dammah                  5
Syncerus c. caffer           5
Hippotragus niger            4
Oryx leucoryx                3
Redunca fulvorufula          2
Tragelaphus spekii           1
Name: species, dtype: int64

In [22]:
data.age.value_counts()

adult       153
subadult     20
young        16
Name: age, dtype: int64

## Motivation of the model assumptions

In [23]:
data.modal.value_counts()

unimodal    150
bimodal      39
Name: modal, dtype: int64

Serial dependence of SC:

In [24]:
def serial_correlation(row):
    scs = str2float(row['scs'])
    if len(scs) < 3:
        return np.nan
    corr = np.corrcoef(scs[:-1], scs[1:])[0, 1]
    return corr

data['scs_serial_corr'] = data.apply(serial_correlation, axis=1)

data['scs_serial_corr'].mean(), data['scs_serial_corr'].sem()

(0.05746445494229766, 0.009950937292967888)

Increase or decrease of SC during night

In [25]:
def regrr(row):
    if len(row['scs']) < 3:
        return np.nan
    scs = str2float(row['scs'])
    xcs = str2float(row['starts'])
    if len(scs) < 3:
        return np.nan
    return np.polyfit(xcs, scs, 1)[0]

nights['scs_reg'] = nights.apply(regrr, axis=1)

In [26]:
nights['scs_reg'].mean() * 60, nights['scs_reg'].sem() * 60 * 60

(2.344833112687312, 11.84788625254998)

Serial dependence of LF

In [27]:
def serial_correlation(row):
    scs = str2float(row['lfs'])
    if len(scs) < 3:
        return np.nan
    corr = np.corrcoef(scs[:-1], scs[1:])[0, 1]
    return corr

data['lfs_serial_corr'] = data.apply(serial_correlation, axis=1)

data['lfs_serial_corr'].mean(), data['lfs_serial_corr'].sem()

(-0.049652033809106884, 0.011504099776625698)

Increase or decrease of LF during night

In [28]:
def regrr(row):
    if len(row['lfs']) < 3:
        return np.nan
    scs = str2float(row['lfs'])
    xcs = str2float(row['starts'])
    if len(scs) < 3:
        return np.nan
    return np.polyfit(xcs, scs, 1)[0]

nights['lfs_reg'] = nights.apply(regrr, axis=1)

nights['lfs_reg'].mean() * 60, nights['lfs_reg'].sem() * 60 * 60

(0.0100388935718708, 0.02983527449348149)

## No significant difference in IBN based regularity measure

In [29]:
def scs_m(row):
    scs = str2float(row['scs'])
    if len(scs) < 3:
        return np.nan
    return np.mean(scs)

def scs_sd(row):
    scs = str2float(row['scs'])
    if len(scs) < 3:
        return np.nan
    return np.std(scs)

data['m'] = data.apply(scs_m, axis=1)
data['s'] = data.apply(scs_sd, axis=1)

In [30]:
data['cv'] = data.s / data.m
data['ibn_cv'] = data.sigma / data.mu

In [31]:
data_ = data.query('ibn_cv < 1.8')

In [32]:
stats.wilcoxon(data_['cv'], data_['ibn_cv'])

WilcoxonResult(statistic=3253.0, pvalue=8.055474215139646e-14)

In [33]:
print('CV:')
print('adult vs. young:', stats.mannwhitneyu(data_.query('age == "adult"')['cv'], data_.query('age == "young"')['cv']))
print('adult vs. subadult:', stats.mannwhitneyu(data_.query('age == "adult"')['cv'], data_.query('age == "subadult"')['cv']))
print('subadult vs. young:', stats.mannwhitneyu(data_.query('age == "subadult"')['cv'], data_.query('age == "young"')['cv']))

CV:
adult vs. young: MannwhitneyuResult(statistic=456.0, pvalue=0.00013041889327624647)
adult vs. subadult: MannwhitneyuResult(statistic=1676.0, pvalue=0.45761387979779977)
subadult vs. young: MannwhitneyuResult(statistic=62.0, pvalue=0.003537936478222105)


In [34]:
print('IBN-CV:')
print('adult vs. young:', stats.mannwhitneyu(data_.query('age == "adult"')['ibn_cv'], data_.query('age == "young"')['ibn_cv']))
print('adult vs. subadult:', stats.mannwhitneyu(data_.query('age == "adult"')['ibn_cv'], data_.query('age == "subadult"')['ibn_cv']))
print('subadult vs. young:', stats.mannwhitneyu(data_.query('age == "subadult"')['ibn_cv'], data_.query('age == "young"')['ibn_cv']))

IBN-CV:
adult vs. young: MannwhitneyuResult(statistic=856.0, pvalue=0.11255842060948171)
adult vs. subadult: MannwhitneyuResult(statistic=1895.0, pvalue=0.07363321908685885)
subadult vs. young: MannwhitneyuResult(statistic=94.0, pvalue=0.06431354959122743)


Higher interruption probabilities in younger animals

In [35]:
print('Pi:')
print('adult vs. young:', stats.mannwhitneyu(data_.query('age == "adult"')['pi'], data_.query('age == "young"')['pi']))
print('adult vs. subadult:', stats.mannwhitneyu(data_.query('age == "adult"')['pi'], data_.query('age == "subadult"')['pi']))
print('subadult vs. young:', stats.mannwhitneyu(data_.query('age == "subadult"')['pi'], data_.query('age == "young"')['pi']))

Pi:
adult vs. young: MannwhitneyuResult(statistic=551.0, pvalue=0.0009879459340609922)
adult vs. subadult: MannwhitneyuResult(statistic=1323.0, pvalue=0.34792167732735235)
subadult vs. young: MannwhitneyuResult(statistic=99.0, pvalue=0.09231058101189886)


In [36]:
(data.query('age == "young"').pi > 0.05).value_counts()

True     14
False     2
Name: pi, dtype: int64

In [37]:
(data.query('age == "subadult"').pi > 0.05).value_counts()

True     15
False     5
Name: pi, dtype: int64

In [38]:
(data.query('age == "adult"').pi > 0.05).value_counts()

True     84
False    69
Name: pi, dtype: int64

In [39]:
stats.chi2_contingency([[14, 15, 84], [2, 5, 69]])

Chi2ContingencyResult(statistic=8.555073426669225, pvalue=0.01387680256303785, dof=2, expected_freq=array([[ 9.56613757, 11.95767196, 91.47619048],
       [ 6.43386243,  8.04232804, 61.52380952]]))

In [40]:
(data.query('modal == "bimodal"').pi > 0.05).value_counts()

True     37
False     2
Name: pi, dtype: int64

In [41]:
(data.query('modal == "unimodal"').pi > 0.05).value_counts()

True     76
False    74
Name: pi, dtype: int64